In [ ]:
import os
os.getcwd()

In [ ]:
root_path = os.getcwd()

In [ ]:
# Import Necessary Libraries
import numpy as np
from datetime import datetime
import pysteps
# Import utility functions from the repository (custom functions for data loading and other functionalities
import utility   

In [ ]:
# Define the data directory for the repository data
data_path = os.path.join(root_path, "Data")
# Define metadata files
metadata_X = utility.get_matadata(os.path.join(data_path, "radarmappatipo.tif"), type="X")
# metadata_C = utility.get_matadata(os.path.join(data_path, "civilpromap.tiff"), type="C")

# Look for Event folder

In [ ]:
# List all folders in the data path
folders = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder))]
folders

In [ ]:
events = [event for event in os.listdir(os.path.join(data_path,'UNICA_SG')) if os.path.isdir(os.path.join(data_path, 'UNICA_SG', event))]
events

In [ ]:
events[3]

# Check the number of file available for analysis

In [ ]:
count_events = [utility.count_files(os.path.join(data_path, 'UNICA_SG', events[i]), 'png') for i in range(len(events))]
count_events

# Load and Process X-Band Radar Data

In [ ]:
# Define parameters for X-band radar data
data_source_X = 'UNICA_SG'
event_subdir = events[3]
f_ext_X = "png"
num_prev_files =9
timestep = 5

In [ ]:
# Choose the date for particular subset
eventdates = [eventdate for eventdate in os.listdir(os.path.join(data_path,'UNICA_SG',event_subdir)) if os.path.isfile(os.path.join(data_path,'UNICA_SG', event_subdir,eventdate))]
eventdates

In [ ]:
date = datetime.strptime('20191222_0820.png', "%Y%m%d_%H%M.png")
date

In [ ]:
# Load X-band radar data
R_X, metadata_X = utility.import_files_by_date(date, data_path, data_source_X,event_subdir, f_ext_X, metadata_X,num_prev_files,0,timestep=timestep)

In [ ]:
# Verify results
print("Radar Data Shape:", R_X.shape)
print("Timestamps:", metadata_X["timestamps"])

# Noise Removal

In [ ]:
# Noise removal using Watershed technique
R_X_clean = np.empty_like(R_X)
for t in range(R_X.shape[0]):
    R_X_clean[t, :, :] = utility.noise_remove(R_X[t, :, :], type="Watershed")

# Data Transfromation

In [ ]:
# Convert Digital Number to Reflectivity (dBZ)
R_X_dbz,metadata_dbz = utility.dn_to_dbz(R_X_clean,metadata_X)
# Convert to rain rate
R_X_R, metadata_R = pysteps.utils.conversion.to_rainrate(R_X_dbz, metadata_dbz)
# Log-transform the data
R_dBR, metadata_dBR = pysteps.utils.transformation.dB_transform(R_X_R, metadata_X, threshold=0.01, zerovalue=-15.0)

In [ ]:
# Print metadata
from pprint import pprint
pprint(metadata_dBR)

# Rain Intensity GIF


In [ ]:
# Create a GIF that loops indefinitely
utility.create_gif(R_X_dbz, metadata_X,duration=5, units="mm/h", title="Precipitation Intensity",loop=True)

# Motion Field Estimation - Lucas Kanade (LK)
## It needs only 3 scans

In [ ]:
# Estimate motion field using Lucas-Kanade optical flow
oflow_method = pysteps.motion.get_method("LK")
V1 = oflow_method(R_dBR[-10:, :, :])

# Visualization

In [ ]:
# Plot precipitation field and motion field
import matplotlib.pyplot as plt
from pysteps.visualization import plot_precip_field, quiver
fig, ax = plt.subplots(figsize=(10, 8))
# Precipitation field
plot_precip_field(
    R_X_dbz[-1, :, :], 
    ptype="intensity", 
    geodata=metadata_dbz, 
    units="mm/h", 
    title= "Lucas Kanade Optical Flow Method of "+ metadata_dbz['timestamps'][2].strftime("%d/%m/%Y %H:%M"),
    ax=ax, 
    colorscale="pysteps"
)
# Motion field (quiver plot)
quiver(V1, geodata=metadata_dbz, step=45, ax=ax)
utility.plot_modification(ax,metadata_dbz)

# Motion Field Estimation- Variational echo tracking (VET) Method
## It needs only  3 scans
### (it will take some time to run the algorithm)

In [ ]:
oflow_method = pysteps.motion.get_method("VET")
V2 = oflow_method(R_dBR[-3:, :, :])

# Plot the motion field
fig, ax = plt.subplots(figsize=(10, 8))
# Precipitation field
plot_precip_field(
    R_X_dbz[-1, :, :], 
    ptype="intensity", 
    geodata=metadata_dbz, 
    units="mm/h", 
    title="VET Method of "+ metadata_dbz['timestamps'][2].strftime("%d/%m/%Y %H:%M"), 
    ax=ax, 
    colorscale="pysteps"
)
quiver(V2, geodata=metadata_dbz, step=45,ax=ax)
utility.plot_modification(ax,metadata_dbz)

# Motion Field Estimation - Dynamic and adaptive radar tracking of storms (DARTS)
## It needs last 10 scans

In [ ]:
oflow_method = pysteps.motion.get_method("DARTS")
R_dBR[~np.isfinite(R_dBR)] = metadata_dBR["zerovalue"]
V3 = oflow_method(R_dBR[-10:,:,:])  # needs longer training sequence

In [ ]:
# Plot the motion field
fig, ax = plt.subplots(figsize=(10, 8))
# Precipitation field
plot_precip_field(
    R_X_dbz[-1, :, :], 
    ptype="intensity", 
    geodata=metadata_dbz, 
    units="mm/h", 
    title="DARTS Method of "+ metadata_dbz['timestamps'][9].strftime("%d/%m/%Y %H:%M"), 
    ax=ax, 
    colorscale="pysteps"
)
quiver(V3, geodata=metadata_dbz, step=45,ax=ax)
utility.plot_modification(ax,metadata_dbz)

# Motion Field Estimation -  Anisotropic Diffusion Method 
## It need only last 2 scans

In [ ]:
oflow_method = pysteps.motion.get_method("proesmans")
R_dBR[~np.isfinite(R_dBR)] = metadata_dBR["zerovalue"]
V4 = oflow_method(R_dBR[-2:, :, :])

In [ ]:
# Plot the motion field
fig, ax = plt.subplots(figsize=(10, 8))
# Precipitation field
plot_precip_field(
    R_X_dbz[-1, :, :], 
    ptype="intensity", 
    geodata=metadata_dbz, 
    units="mm/h", 
    title="Anisotropic Diffusion Method of "+ metadata_dbz['timestamps'][2].strftime("%d/%m/%Y %H:%M"), 
    ax=ax, 
    colorscale="pysteps"
)
quiver(V4, geodata=metadata_dbz, step=45,ax=ax)
utility.plot_modification(ax,metadata_dbz)